In [ ]:
import requests
import pandas as pd
from bokeh.layouts import column, row
from prophet import Prophet
from bokeh.plotting import figure, show
from bokeh.models import BoxZoomTool, WheelZoomTool, ResetTool, PanTool, SaveTool, BoxSelectTool, TapTool, CrosshairTool 
from bokeh.io import output_notebook
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import Select, TextInput, MultiSelect, CheckboxGroup, Checkbox
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from pydlm import dlm, trend, seasonality

In [ ]:
url = "https://api.binance.com/api/v3/klines"

source_actual = ColumnDataSource()
source_forecast = ColumnDataSource()

hover_tool = HoverTool(tooltips=[("Date", "@ds{%F}"), ("Actual Price", "@y"), ("Forecasted Price", "@yhat")], formatters={"@ds": "datetime"})
symbol_select = TextInput(value="BTCUSDT", title="Symbol:")
interval_select = Select(title="Interval", options=["1d", "3d", "1w"], value="3d")
predict_period_input = TextInput(value="30", title="Prediction Periods:")
seasonality_mode_select = Select(title="Seasonality Mode", options=["additive", "multiplicative"], value="additive")
changepoint_prior_scale_input = TextInput(value="0.05", title="Changepoint Prior Scale:")

def get_data(symbol, interval, limit):
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

def update_data(attr, old, new):
    symbol = symbol_select.value
    interval = interval_select.value
    periods = int(predict_period_input.value)
    df = get_data(symbol, interval, 1000)
    
    prophet_df = df[["timestamp", "close"]].rename(columns={"timestamp": "ds", "close": "y"})
    prophet_df["y"] = pd.to_numeric(prophet_df["y"])
    model = Prophet(seasonality_mode=seasonality_mode_select.value, changepoint_prior_scale=float(changepoint_prior_scale_input.value))
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)

    source_actual.data = prophet_df
    source_forecast.data = forecast

# Plot 
p = figure(title=f"Price Forecast", x_axis_label="Date", x_axis_type="datetime", width=1600, tools=[hover_tool, BoxZoomTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool(), BoxSelectTool(), TapTool(), CrosshairTool()])
p.line(source=source_actual, x="ds", y="y", color="blue", legend_label="Actual Price")
p.line(source=source_forecast, x="ds", y="yhat", color="red", legend_label="Forecasted Price")
p.add_tools(hover_tool)

symbol_select.on_change("value", update_data) 
interval_select.on_change("value", update_data)
predict_period_input.on_change("value", update_data)
seasonality_mode_select.on_change("value", update_data)
changepoint_prior_scale_input.on_change("value", update_data)

layout = column(row(symbol_select, interval_select, predict_period_input, seasonality_mode_select, changepoint_prior_scale_input), p)
curdoc().add_root(layout)
